In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import cv2
import os
from os import listdir
from os.path import isfile, isdir, join

import random
from PIL import Image
import pickle
import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
import gc

torch.cuda.empty_cache()

SIZE = 256
size = 224
BATCH_SIZE = 32
EPOCH = 15
PATIENCE = 4
LR_RATE = 0.005

wb = 0
if wb == 1:
    wandb.init(project="Tim_test", name = 'effnet-b4 ensemble 233131113' , entity="ct_01",
               config = {  "learning_rate": LR_RATE,
                "epoch": EPOCH,
                "batch_size": BATCH_SIZE,
                "resized": size,
                "model": "effnet-b4 with pretrained",
                "augment": "2:3:3, 1:3:1, 1:1:3",
                "weight": "1:0.5:0.5"})
    
    #wandb.config = {  "learning_rate": 0.001,
     #           "epoch": EPOCH,
      #          "batch_size": BATCH_SIZE,
       #         "resized": size }

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#aa

# fixed seed

In [3]:
seed = 887
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# load picture function

In [4]:
from torchvision import transforms
from torchvision.transforms import functional as TF


def load_picture_data(path):
    path0 = path
    files = listdir(path0)
    files.sort()
    #print(files)
    #stop
    all_images = []
    imges_var = []
    label = []
    cnt=0
    for every_image in files:
        cnt += 1
        #if cnt > 300: break
        fullpath = join(path0, every_image)
        if '.jpg' in every_image:
            #print('haha')
            #im = Image.open(fullpath)
            img = cv2.imread(fullpath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #img = ((img-img.min())/(img.max()-img.min())*255).astype(np.uint8)  # min max
            #print(img.dtype)
            #new_img = img_transform(im)
            
            ######### blurring#####################################
            transeImg = cv2.Laplacian(img, cv2.CV_64F)#############
            imgVar = transeImg.var() ##################
            imges_var.append(imgVar)
            ######################################
            #new_img = im.resize((size, size), Image.BILINEAR)  # change to the same size
            
            ###############
            # use CLAHE####
            ###############
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            #img = clahe.apply(img)
            
            #res = np.hstack((img, cl1))
            #if cnt <= 5: cv2.imwrite(every_image, res)

            img = np.array(img)  # image类转 numpy
            
            
            
            if len(img.shape) == 2:  # gray
                #print('here', path)
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                # print(every_image, img.shape)
                
            all_images.append(img)
            #imges_var.append(imageVar)
           
                
    print('load data over')

    data = np.array(all_images)
    #print(data.shape)
    #label = pd.read_csv('_info.csv')

    return data, np.array(imges_var)

# augment function

In [5]:
def augment(x_data, is_train, augment_flag):
    img_transform = transforms.Compose([
        #transforms.Resize((SIZE, SIZE)),
        transforms.CenterCrop((512, 384)),
        transforms.Resize((size, size), Image.BILINEAR),
        #transforms.RandomResizedCrop(size, scale=(0.5, 1.0)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
    ])
    img_transform2 = transforms.Compose([
        #transforms.RandomHorizontalFlip(),
        #transforms.Resize((SIZE, SIZE)),
        transforms.RandomRotation(5),
        transforms.RandomResizedCrop(size), #, scale=(0.2, 1.0)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_transform3 = transforms.Compose([
        #transforms.Resize((SIZE, SIZE)),
        transforms.RandomResizedCrop(size),
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    
    other_img_transform = transforms.Compose([
        #transforms.Resize((SIZE, SIZE)),
        transforms.Resize((size, size), Image.BILINEAR),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    x_data_new=[]
    
    for i in range(len(x_data)):
        #print(x_train[i].shape)
        tmp = Image.fromarray(x_data[i])
        #x_train[i] = img_transform(tmp)
        if is_train==1 and augment_flag==1: x_data_new.append(np.array(img_transform(tmp)))
        elif is_train==2 and augment_flag==1: x_data_new.append(np.array(img_transform2(tmp)))
        elif is_train==3 and augment_flag==1: x_data_new.append(np.array(img_transform3(tmp)))
        #elif is_train and augment_flag==2: x_data_new.append(np.array(img_transform2(tmp)))
        #if is_train: x_data_new.append(np.array(img_transform(tmp))/255)
        else: x_data_new.append(np.array(other_img_transform(tmp)))
    #print(x_data_new[0])        
    return np.array(x_data_new)

# Main load data

In [6]:
from sklearn.model_selection import train_test_split

if __name__ == '__main__':
    #####################3
    
    #################
    
    data = 0
    valid_data = 0
    label = 0
    
    #train_path = '../input/case2-data/resized_train_data/resized_train_data'
    #valid_path = '../input/case2-data/resized_valid_data'
    
    train_path = '../input/stacking-data/stacking_data/train'
    valid_path = '../input/stacking-data/stacking_data/valid'
    
    data, data_var = load_picture_data(train_path)
    valid_data, valid_data_var = load_picture_data(valid_path)
    
    
    #idx = np.argsort(data_var)
    #print(np.sort(data_var)[0:50])
    
    valid_dir = listdir(valid_path)
    valid_dir.sort()

    
    label = pd.read_csv('../input/case2-data/_info.csv')
    #for i in range(10):
     #   print(label.loc[idx[i]])
    label = label.drop(columns='FileID')
    label = np.array(label)
    label = np.argmax(label, axis=1)
      
    print(data.shape, label.shape)
    
    #print(label)

load data over
load data over
(1200, 512, 512, 3) (1200,)


# split & to tensor

In [7]:
    
    
    x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, stratify=label, random_state=seed)
    #x_train, y_train = data.copy(), label.copy()
    
    add_ill_data_id = np.where(y_train>0) # augment on typical/atypical data
    #print(add_ill_data_id)
    add_ill_data = x_train[add_ill_data_id]
    add_ill_label = y_train[add_ill_data_id]
    

    print(np.unique(y_train, return_counts=True))
    print(np.unique(y_test, return_counts=True))
    
    print(add_ill_data.shape, add_ill_label.shape)
    
    
    #print(np.unique(y_valid, return_counts=True))
    #st

(array([0, 1, 2]), array([280, 280, 280]))
(array([0, 1, 2]), array([120, 120, 120]))
(560, 512, 512, 3) (560,)


# add typical/atypical data

In [8]:
    x_train_original = x_train.copy()
    y_train_original = y_train.copy()
    
    x_train_typical = x_train.copy()
    y_train_typical = y_train.copy()
    x_train_atypical = x_train.copy()
    y_train_atypical = y_train.copy()
    
    
    
    typical_data_id = np.where(y_train==1) # more augment on atypical data
    atypical_data_id = np.where(y_train>1) # more augment on atypical data
    
    typical_data = x_train_typical[typical_data_id]
    atypical_data = x_train_atypical[atypical_data_id]
    
    typical_label = y_train_typical[typical_data_id]
    atypical_label = y_train_atypical[atypical_data_id]
    
    
    #non_atypical_id = np.where(y_train<2)
    #non_typical_id = np.where(y_train!=1)
    #x_train_typical = x_train_typical[non_atypical_id] # only has 0, 1
    #y_train_typical = y_train_typical[non_atypical_id] # only has 0, 1
    #x_train_atypical = x_train_atypical[non_typical_id] # only has 0, 2
    #y_train_atypical = y_train_atypical[non_typical_id] # only has 0, 2
    
    print(typical_data.shape, atypical_data.shape)
    print(typical_label.shape, atypical_label.shape)
    #print(typical_label)

(280, 512, 512, 3) (280, 512, 512, 3)
(280,) (280,)


# rotation & augment(choose)

In [9]:
    
    #print(x_train.shape)
    augment_flag = 1  ###### change this one
    
    x_train_aug = augment(x_train, 1, augment_flag)
    #x_train_aug2 = augment(x_train, 3, augment_flag)
    
    
    x_train = augment(x_train, 1, 0)
    x_test = augment(x_test, 0, 0)
    #x_valid = augment(x_valid, 0, 0)
    valid_data = augment(valid_data, 0, 0)
    
    add_ill_data_first = augment(add_ill_data, 2, augment_flag) # augment on illness data
    #add_ill_data_second = augment(add_ill_data, 3, augment_flag) # augment on illness data
    #atypical_data_first = augment(atypical_data, 3, augment_flag) # augment on atypical data
    
    
    y_train_aug = y_train.copy()
    
    if augment_flag == 1:    # *3 datas
        x_train = np.concatenate([x_train, x_train_aug])
        y_train = np.concatenate([y_train, y_train_aug])
        
        #x_train = np.concatenate([x_train, x_train_aug2])
        #y_train = np.concatenate([y_train, y_train_aug])
        
        x_train = np.concatenate([x_train, add_ill_data_first]) # add illness data(augment)
        y_train = np.concatenate([y_train, add_ill_label])
        #x_train = np.concatenate([x_train, atypical_data_first]) # add atypical data
        #y_train = np.concatenate([y_train, atypical_label])
            
            
    np.random.seed(seed)
    np.random.shuffle(x_train)
    np.random.seed(seed)
    np.random.shuffle(y_train)
    
    print(x_train.shape, y_train.shape)
    #stop

/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


(2240, 3, 224, 224) (2240,)


# augment on typical/atypical

In [10]:
    augment_flag = 1  ###### change this one
    
    x_train_typical_aug = augment(typical_data, 1, augment_flag)
    x_train_atypical_aug = augment(atypical_data, 1, augment_flag)
 
    another_x_train_typical_aug = augment(typical_data, 2, augment_flag)
    another_x_train_atypical_aug = augment(atypical_data, 2, augment_flag)
    
    #third_x_train_typical_aug = augment(typical_data, 3, augment_flag)
    #third_x_train_atypical_aug = augment(atypical_data, 3, augment_flag)
    
    #x_train_original_ = augment(x_train_original, 1, 0)
    x_train_typical = augment(x_train_typical, 1, 0)
    x_train_atypical = augment(x_train_atypical, 1, 0)
    
    #y_train_aug = y_train.copy()
    
    if augment_flag == 1:    # *3 datas
        x_train_typical = np.concatenate([x_train_typical, x_train_typical_aug, another_x_train_typical_aug])
        x_train_atypical = np.concatenate([x_train_atypical, x_train_atypical_aug, another_x_train_atypical_aug])
        
        y_train_typical = np.concatenate([y_train_typical, typical_label, typical_label])
        y_train_atypical = np.concatenate([y_train_atypical, atypical_label, atypical_label])
        
        ###################### 1:1:4 & 1:4:1 aug
        #x_train_atypical = np.concatenate([x_train_atypical, third_x_train_atypical_aug])
        #y_train_atypical = np.concatenate([y_train_atypical, atypical_label])
        #x_train_typical = np.concatenate([x_train_typical, third_x_train_typical_aug])
        #y_train_typical = np.concatenate([y_train_typical, typical_label])
        
    
    np.random.seed(seed)
    np.random.shuffle(x_train_typical)
    np.random.seed(seed)
    np.random.shuffle(x_train_atypical)
    np.random.seed(seed)
    np.random.shuffle(y_train_typical)
    np.random.seed(seed)
    np.random.shuffle(y_train_atypical)
    
    
    print(x_train_typical.shape, y_train_typical.shape)
    print(x_train_atypical.shape, y_train_atypical.shape)
    #for item in y_train_atypical:
     #   print(item, end=' ')
    #print(y_train_typical)

(1400, 3, 224, 224) (1400,)
(1400, 3, 224, 224) (1400,)


# to tensor

In [11]:

    x_train = torch.tensor(x_train).to(dtype=torch.float16)
    x_test = torch.tensor(x_test).to(dtype=torch.float16)
    #x_valid = torch.tensor(x_valid).to(dtype=torch.float16)
    valid_data = torch.tensor(valid_data).to(dtype=torch.float16)

    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)
    #y_valid = torch.tensor(y_valid, dtype=torch.long)

    #x_train = x_train.permute(0, 3, 1, 2)  # shape:[10000, 3, 32, 32]
    #x_test = x_test.permute(0, 3, 1, 2)
    #x_valid = x_valid.permute(0, 3, 1, 2)
    #valid_data = valid_data.permute(0, 3, 1, 2)
    
    print(x_train.shape)
    print(x_test.shape)
    #print(x_valid.shape)
    print(valid_data.shape)

torch.Size([2240, 3, 224, 224])
torch.Size([360, 3, 224, 224])
torch.Size([150, 3, 224, 224])


# to tensor (ty/aty..)

In [12]:
    x_train_typical = torch.tensor(x_train_typical).to(dtype=torch.float16)
    x_train_atypical = torch.tensor(x_train_atypical).to(dtype=torch.float16)

    y_train_typical = torch.tensor(y_train_typical, dtype=torch.long)
    y_train_atypical = torch.tensor(y_train_atypical, dtype=torch.long)
    
    print(x_train_typical.shape)
    print(x_train_atypical.shape)
    print(y_train_typical.shape)
    print(y_train_atypical.shape)

torch.Size([1400, 3, 224, 224])
torch.Size([1400, 3, 224, 224])
torch.Size([1400])
torch.Size([1400])


# to loader

In [13]:
    trainloader = torch.utils.data.DataLoader(x_train, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    train_labels_loader = torch.utils.data.DataLoader(y_train, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    testloader = torch.utils.data.DataLoader(x_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    test_labels_loader = torch.utils.data.DataLoader(y_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    #validloader = torch.utils.data.DataLoader(x_valid, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    #valid_labels_loader = torch.utils.data.DataLoader(y_valid, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    submitloader = torch.utils.data.DataLoader(valid_data, batch_size=150, shuffle=False, num_workers=2)

# to loader (typ/atyp)

In [14]:
    trainloader_typical = torch.utils.data.DataLoader(x_train_typical, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    trainloader_atypical = torch.utils.data.DataLoader(x_train_atypical, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    train_labels_loader_typical = torch.utils.data.DataLoader(y_train_typical, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    train_labels_loader_atypical = torch.utils.data.DataLoader(y_train_atypical, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [15]:
#!pip install pretrainedmodels
#!pip install albumentations

!pip install --upgrade efficientnet-pytorch
from efficientnet_pytorch import EfficientNet

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=aa2be28f9c5f0b1fde1018d2957da40c9df8b201b4b1ba82b1cc09f89ee22edc
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


# model & param

In [16]:

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    
    #model = models.resnet101(pretrained=True).to(device, dtype=torch.float16)
    
    #model = models.resnet152(pretrained=True).to(device, dtype=torch.float16)
    #model = models.efficientnet_b7(pretrained=True).to(device, dtype=torch.float16)
    model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=3).to(device, dtype=torch.float16)
    
    #num_ftrs = model.fc.in_features
    #model.fc = nn.Linear(num_ftrs, 3).to(device, dtype=torch.float16)
    #model.fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(num_ftrs, 3)).to(device, dtype=torch.float16)
    #print(model.fc)
    
    model.eval()
    #if wb == 1: wandb.log({"model": 'resnet101 with pretrained'})
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=LR_RATE, momentum=0.9, weight_decay=5e-4)
    #optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)


cuda


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth


  0%|          | 0.00/74.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b4


# another two models

In [17]:

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    
    model_typical = EfficientNet.from_pretrained('efficientnet-b4', num_classes=3).to(device, dtype=torch.float16)
    model_atypical = EfficientNet.from_pretrained('efficientnet-b4', num_classes=3).to(device, dtype=torch.float16)
    
    #model_typical = models.resnet101(pretrained=True).to(device, dtype=torch.float16)
    #model_atypical = models.resnet101(pretrained=True).to(device, dtype=torch.float16)
    
    #num_ftrs_typical = model_typical.fc.in_features
    #num_ftrs_atypical = model_atypical.fc.in_features
    #model_typical.fc = nn.Linear(num_ftrs_typical, 3).to(device, dtype=torch.float16)
    #model_atypical.fc = nn.Linear(num_ftrs_atypical, 3).to(device, dtype=torch.float16)
    
    model_typical.eval()
    model_atypical.eval()
    
    criterion_typical = nn.CrossEntropyLoss()
    optimizer_typical = optim.SGD(model_typical.parameters(), lr=LR_RATE, momentum=0.9, weight_decay=5e-4)
    criterion_atypical = nn.CrossEntropyLoss()
    optimizer_atypical = optim.SGD(model_atypical.parameters(), lr=LR_RATE, momentum=0.9, weight_decay=5e-4)
    

cuda
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4


# training

In [18]:
    final_predict=[]
    
    for epoch in range(0, EPOCH):
        if epoch == 10:
            optimizer = optim.SGD(model.parameters(), lr=LR_RATE/2, momentum=0.9, weight_decay=5e-4)
            optimizer_typical = optim.SGD(model_typical.parameters(), lr=LR_RATE/2, momentum=0.9, weight_decay=5e-4)
            optimizer_atypical = optim.SGD(model_atypical.parameters(), lr=LR_RATE/2, momentum=0.9, weight_decay=5e-4)
        #print(optimizer)
        final_predict=[]
        print('\n---------------------------------------------')
        model.train()
        sum_loss = 0.0
        correct = 0.0
        total = 0.0
        for (i, data), labels_data in zip(enumerate(trainloader), train_labels_loader):
            length = len(trainloader)
            inputs, labels = data, labels_data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # forward + backward
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            #
            sum_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            #print(outputs, predicted)
            #fk
            total += labels.size(0)
            correct += predicted.eq(labels.data).cpu().sum()
            
            
            # print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
            # % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
        print('Model1: Epoch: %d, Loss: %.03f, Train Accuray: %.3f' % (epoch + 1, sum_loss / (i + 1), correct / total))
        #
        train_acc = correct / total
        train_loss = sum_loss/(i + 1)
        
        ############################################################################################
        #### model_typical & model_atypical
        ###############################################
        model_typical.train()
        sum_loss = 0.0
        correct = 0.0
        total = 0.0
        for (i, data), labels_data in zip(enumerate(trainloader_typical), train_labels_loader_typical):
            length = len(trainloader_typical)
            inputs, labels = data, labels_data
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer_typical.zero_grad()

            # forward + backward
            outputs = model_typical(inputs)
            #print(inputs.shape, outputs.shape, labels.shape)
            #print(outputs)
            loss = criterion_typical(outputs, labels)
            loss.backward()
            optimizer_typical.step()

            #
            sum_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            #print(outputs, predicted)
            #fk
            total += labels.size(0)
            correct += predicted.eq(labels.data).cpu().sum()
            
            
            # print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
            # % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
        print('Model2: Epoch: %d, Loss: %.03f, Train Accuray: %.3f' % (epoch + 1, sum_loss / (i + 1), correct / total))
        #
        train_acc_typical = correct / total
        train_loss_typical = sum_loss/(i + 1)
        ######################################
        ##model 3
        model_atypical.train()
        sum_loss = 0.0
        correct = 0.0
        total = 0.0
        for (i, data), labels_data in zip(enumerate(trainloader_atypical), train_labels_loader_atypical):
            length = len(trainloader_atypical)
            inputs, labels = data, labels_data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer_atypical.zero_grad()

            # forward + backward
            outputs = model_atypical(inputs)
            
            loss = criterion_atypical(outputs, labels)
            loss.backward()
            optimizer_atypical.step()

            #
            sum_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            #print(outputs, predicted)
            #fk
            total += labels.size(0)
            correct += predicted.eq(labels.data).cpu().sum()
            
            
            # print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
            # % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
        print('Model3: Epoch: %d, Loss: %.03f, Train Accuray: %.3f' % (epoch + 1, sum_loss / (i + 1), correct / total))
        #
        train_acc_atypical = correct / total
        train_loss_atypical = sum_loss/(i + 1)
        
        # Optional
        ###########################
        # test accuracy
        ###########################
        
        with torch.no_grad():
            correct = 0
            total = 0
            test_losses = []
            for data, label_data in zip(testloader, test_labels_loader):
                model.eval()
                model_typical.eval()
                model_atypical.eval()
                images, labels = data, label_data
                images, labels = images.to(device), labels.to(device)
                
                outputs1 = model(images)
                outputs2 = model_typical(images)
                outputs3 = model_atypical(images)
                
                outputs_softmax1 = torch.nn.functional.softmax(outputs1, dim=1)
                outputs_softmax2 = torch.nn.functional.softmax(outputs2, dim=1)
                outputs_softmax3 = torch.nn.functional.softmax(outputs3, dim=1)
                
                
                all_outputs = (outputs_softmax1 + outputs_softmax2*0.5 + outputs_softmax3*0.5)/3  # add weight on model 2,3
                
                loss = criterion(all_outputs, labels)
                
                test_losses.append(loss.item())
                _, predicted = torch.max(all_outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
                #print(predicted.cpu().detach().numpy())
                final_predict = np.concatenate([final_predict, predicted.cpu().detach().numpy()])
            #print(final_predict)
            final_predict = np.array(final_predict)    
            test_f1 = f1_score(y_test, final_predict, average='macro')
            test_loss = np.average(test_losses)
            
            print('Test F1 score:%.3f, Test Loss:%.3f, Test Accuracy：%.3f' % (test_f1, test_loss, correct / total))
            acc = 100. * correct / total

            #if acc > best_acc:
             #   best_acc = acc
        test_acc = correct / total
        test_loss = test_loss
        
        
        if wb == 1:
            wandb.log({"Train Accuracy": train_acc,
                        "Train loss": train_loss,
                        "Test Accuracy": test_acc,
                        "Test F1 score": test_f1,
                        "Test Loss": test_loss,
                      })
                        #"Valid Accuracy": valid_acc,
                        #"Valid loss": valid_loss})
    


---------------------------------------------
Model1: Epoch: 1, Loss: 1.017, Train Accuray: 0.471
Model2: Epoch: 1, Loss: 0.916, Train Accuray: 0.573
Model3: Epoch: 1, Loss: 0.947, Train Accuray: 0.571
Test F1 score:0.427, Test Loss:1.076, Test Accuracy：0.472

---------------------------------------------
Model1: Epoch: 2, Loss: 0.820, Train Accuray: 0.624
Model2: Epoch: 2, Loss: 0.659, Train Accuray: 0.713
Model3: Epoch: 2, Loss: 0.702, Train Accuray: 0.687
Test F1 score:0.510, Test Loss:1.056, Test Accuracy：0.533

---------------------------------------------
Model1: Epoch: 3, Loss: 0.616, Train Accuray: 0.755
Model2: Epoch: 3, Loss: 0.520, Train Accuray: 0.786
Model3: Epoch: 3, Loss: 0.518, Train Accuray: 0.798
Test F1 score:0.541, Test Loss:1.035, Test Accuracy：0.561

---------------------------------------------
Model1: Epoch: 4, Loss: 0.378, Train Accuray: 0.864
Model2: Epoch: 4, Loss: 0.374, Train Accuray: 0.851
Model3: Epoch: 4, Loss: 0.356, Train Accuray: 0.872
Test F1 score:

In [19]:

import pickle

#with open('../input/models/the_model1.sav', 'rb') as f:
#    model = pickle.load(f)
#with open('../input/models/the_model2.sav', 'rb') as f:
#    model_typical = pickle.load(f)
#with open('../input/models/the_model3.sav', 'rb') as f:
#    model_atypical = pickle.load(f)
    
final_predict=[]

with torch.no_grad():
    correct = 0
    total = 0
    test_losses = []
    for data, label_data in zip(testloader, test_labels_loader):
        model.eval()
        model_typical.eval()
        model_atypical.eval()
        images, labels = data, label_data
        images, labels = images.to(device), labels.to(device)
                
        outputs1 = model(images)
        outputs2 = model_typical(images)
        outputs3 = model_atypical(images)
                
        outputs_softmax1 = torch.nn.functional.softmax(outputs1, dim=1)
        outputs_softmax2 = torch.nn.functional.softmax(outputs2, dim=1)
        outputs_softmax3 = torch.nn.functional.softmax(outputs3, dim=1)
                
                
        all_outputs = (outputs_softmax1*2 + outputs_softmax2 + outputs_softmax3)/3  # add weight on model 2,3
        #all_outputs[:,2]*=1.25
        loss = criterion(all_outputs, labels)
                
        test_losses.append(loss.item())
        _, predicted = torch.max(all_outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
                #print(predicted.cpu().detach().numpy())
        final_predict = np.concatenate([final_predict, predicted.cpu().detach().numpy()])
            #print(final_predict)
    final_predict = np.array(final_predict)    
    test_f1 = f1_score(y_test, final_predict, average='macro')
    test_loss = np.average(test_losses)
            
    print('Test F1 score:%.3f, Test Loss:%.3f, Test Accuracy：%.3f' % (test_f1, test_loss, correct / total))
    acc = 100. * correct / total

            #if acc > best_acc:
             #   best_acc = acc
test_acc = correct / total
test_loss = test_loss
    


Test F1 score:0.640, Test Loss:0.908, Test Accuracy：0.642


# confusion matrix

In [20]:
import pickle


filename = 'the_model1.sav'
pickle.dump(model, open(filename, 'wb'))
filename2 = 'the_model2.sav'
pickle.dump(model_typical, open(filename2, 'wb'))
filename3 = 'the_model3.sav'
pickle.dump(model_atypical, open(filename3, 'wb'))

#print(final_predict+final_predict_another)
#print(final_predict_another)

matrix = confusion_matrix(y_test, final_predict)
print(matrix)

if wb==1:
    class_labels=['0_Negative', '1_Typical', '2_Atypical']
    #wandb.init()
    wandb.log({"Confusion Matrix": wandb.sklearn.plot_confusion_matrix(
        y_test, final_predict,
        labels=class_labels ) })
            #class_names=['Negative', 'Typical', 'Atypical']) })

[[89  9 22]
 [16 76 28]
 [23 31 66]]


# generate result to submit

In [21]:
    #if wb == 1: wandb.watch(model)
    predict = 0
    cnt=0
    #print(len(submitloader))
    with torch.no_grad():
        for data in submitloader:
            model.eval()
            model_typical.eval()
            model_atypical.eval()
            
            images = data
            images = images.to(device)
            outputs1 = model(images)
            outputs2 = model_typical(images)
            outputs3 = model_atypical(images)
            
            outputs_softmax1 = torch.nn.functional.softmax(outputs1, dim=1)
            outputs_softmax2 = torch.nn.functional.softmax(outputs2, dim=1)
            outputs_softmax3 = torch.nn.functional.softmax(outputs3, dim=1)
            
            all_outputs = (outputs_softmax1*2 + outputs_softmax2 + outputs_softmax3)/3  # add weight on model 2,3
            
            cnt+=1
            print(cnt)
    
    #valid_data = valid_data.to(device)
    #print(valid_data)
    #outputs = model(valid_data)#.cpu().detach().numpy()
    #print(outputs)
            _, predict = torch.max(all_outputs.data, 1)
    
            #print(predict, predict2)
    
    
            
    print(predict)
    #valid_dir = listdir('../input/case2-data/resized_valid_data/')
    #valid_dir.sort()
    
    df_submit = pd.DataFrame(data={'FileID': valid_dir, 'Type': predict.cpu().detach().numpy()})
    #df_submit = pd.DataFrame(data={'FileID': valid_dir, 'Type': merge_predict})
    df_submit['FileID'] = df_submit['FileID'].str.replace('.jpg', '')
    df_submit['Type'] = df_submit['Type'].replace(0, 'Negative')
    df_submit['Type'] = df_submit['Type'].replace(1, 'Typical')
    df_submit['Type'] = df_submit['Type'].replace(2, 'Atypical')

    print(df_submit['Type'].value_counts())
    df_submit.to_csv('./submit.csv', index=False)
    

1
tensor([2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 2, 0, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 1, 0,
        0, 0, 0, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 2, 0, 1, 1, 1,
        0, 2, 1, 0, 1, 0, 2, 0, 2, 2, 1, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1,
        0, 1, 2, 0, 0, 0, 0, 1, 1, 0, 0, 2, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2,
        1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 0, 2, 0, 1, 1, 0, 0, 1, 2, 2, 0, 2, 2, 0,
        0, 2, 0, 1, 0, 1], device='cuda:0')
Negative    58
Typical     52
Atypical    40
Name: Type, dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: The default value of regex will change from True to False in a future version.


In [22]:
if wb == 1:
    run = wandb.init(reinit=True)
    run.finish()